In [2]:
''' Постановка задачи (предсказание цены на квартиру)
    Обработка данных
        Знакомство с данными
        Поиск утечек
        Обработка пропущенных значений
        Обработка кателгориальных значений
        Поиск выбросов
    Применение KNN на полученных данных '''

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import KFold # Помогает избежать утечек
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer # Для создания собственных метрик
from tqdm import tqdm

data = pd.read_csv('melb_data.csv')
X = data.drop(columns='Price') # Удаленик признака, который мы предсказываем
y = data['Price']    #Признак, который мы предсказываем
X = X.drop(columns='Method') # Удаление категориальног признака
# X.isna().sum() - просмотр количества пустых значений по колонкам
# X.loc[X['Car'].isna(), 'Car'] Указание значение строки пустой в столбце Car
X.loc[X['Car'].isna(), 'Car'] = X['Car'].mean() # Подставление среднего значения по колонке в пустые
X = X.drop(columns = 'BuildingArea') # Удаление колонки с большим количеством пустых ячеек
X = X.drop(columns = 'CouncilArea')
X['YearBuiltWasMissing'] = X['YearBuilt'].isna() # Добавление нового столбца, в котором указаны строки, где год постройки здания отсутствует
X.loc[X['YearBuilt'].isna(), 'YearBuilt'] = X['YearBuilt'].mean() # Подстановка среднего года постройки в пустные ячейки
# list(zip(X.columns, X.dtypes)) Типы данных по столбцам
suburb_counts = X['Suburb'].value_counts() # Подсчет уникальных значений по столбцу
is_rare_suburb = X['Suburb'].apply(lambda x: suburb_counts[x] < 200) # Замена значений, которые встречаются реже 200 раз в таблице на одно значение
X.loc[is_rare_suburb, 'Suburb'] = 'RareSuburb'
X = X.drop(columns = ['Address', 'SellerG', 'Date', 'Postcode'])
X = pd.get_dummies(X, columns = ['Suburb', 'Type', 'Regionname', 'YearBuiltWasMissing']) # One-hot-encoding
''' Cоздаём 12 графиков в окне 20, 20, команда subplots возвращает (фигуру, оси в виде матрицы). 
    Далее выбираем только оси, вытягиваем их в один массив командой flatten(). 
    В результате получаем массив из 12 объектов-графиков. В цикле выбираем 1 ось из массива осей, 
    1 числовую переменную из массива числовых переменных 
    и строим по ней бокс с усами на выбранной оси'''
# for ax, num_col_name in zip(plt.subplots(4, 3, figsize=(20,20))[1].flatten(), ['Rooms', 'Distance', 'Bedroom2', 'Bathroom', 'Car', 'Landsize', 'YearBuilt', 'Lattitude', 'Longtitude',  'Propertycount']):
#     ax.set_title(num_col_name)
#     ax.boxplot(X[num_col_name]) # Выбрасывать отличающиеся значения не будем, так как алгоритм KNN устойчив к этим выбросам
''' На последок, мы проведем стандартизацию признаков, про которую говорили раньше.
    Сейчас мы сдлеаем очень плохую и неправильную ошибку. 
    Мы посчитаем среднее и дисперсси признаков по всему датасету, а не только по обучающей части.
    Как вы помните, при кроссвалидации мы выделяем обучающую и валидационную часть только в самом конце, 
    поэтмоу проведя стандартизацию сейчас мы создадим утечку данных. 
    Чтобы этой утечки не допускать нужно использовать sklearn.piplines или пользоваться первым метдом 
    проведение кросс валидации, про который мы поговорим чреез KFold.'''
scaler = StandardScaler() # Происходит утечка, но не большая, так как в данном случае это не критично
X = scaler.fit_transform(X)
knn = KNeighborsRegressor(5)
cv = KFold(n_splits=5)  # Количество частей при кросс-валидации
errors = []

for train_idx, val_idx in tqdm(cv.split(X)):  # tqdm показывает как проходит операция в программе, например в цикле
    X_train, y_train = X[train_idx], y[train_idx]
    X_val, y_val = X[val_idx], y[val_idx]
    
    knn.fit(X_train, y_train)  # Можно вызывать несколько раз, каждый раз модель будет переобучаться
    pred_val = knn.predict(X_val)
    errors.append(
        mean_squared_error(y_val, pred_val)
    )

print()
print('Metrics')
print(errors)
print('RMSE=', np.mean(errors) ** 0.5) # Квадрат средней ошибки

5it [00:01,  3.27it/s]


Metrics
[152217947755.374, 139816382680.09613, 151517859886.62695, 125179488503.45377, 140351366344.85535]
RMSE= 376585.46046559105


In [5]:
scores = cross_val_score(knn, X, y, cv = 5) # Возвращает значение метрики, означает, насколько хорошо работает модель
scores

array([0.65408053, 0.67063013, 0.64918479, 0.63602139, 0.63816684])

In [9]:
scorer = make_scorer(lambda y_true, y_pred: mean_squared_error(y_true, y_pred), greater_is_better=False) # greater_is_better (сохраняет свойство, что чем выше значение метрики, тем лучше) - чем лучше модель, тем выше средний квадрат ошибки 
errors = cross_val_score(knn, X, y, cv = 5, scoring=scorer)

print()
print('Metrics')
print(errors)
print('RMSE=', np.mean(-errors) ** 0.5) # Квадрат средней ошибки


Metrics
[-1.52217948e+11 -1.39816383e+11 -1.51517860e+11 -1.25179489e+11
 -1.40351366e+11]
RMSE= 376585.46046559105
